In [2]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
import time
import pdb
import bdb  # Added to handle BdbQ

In [3]:
if 1: # Create SQLAlchemy engine
from dotenv import load_dotenv
import os
load_dotenv()

connection_string = (
    f"mssql+pyodbc://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}"
    f"@{os.getenv('DB_SERVER')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    f"?driver=ODBC+Driver+17+for+SQL+Server"
    f"&TrustServerCertificate=yes"
    f"&Connection+Timeout=5"
)
    engine = create_engine(connection_string)

In [17]:
# Query all rows from the US_HY_NVR_TEST_2023_2025_REDO table
live_query = """
    SELECT [REPORT_YEAR_WEEK]
      ,[INV_STATE_ABBRV]
      ,[INV_TOWN_NAME]
      ,[DAYS_LSTD_BFR_SLD]
      ,[SRC_VEH_FIRST_SCRAPED_DT]
      ,[TRIM_DESCRIPTION]
      ,[NVI_OWNSHP_DT]
      ,[NVI_EFCTV_START_DT]
      ,[NVI_CENSUS_TRACT]
      ,[NVI_RPT_YYYYMM]
      ,[SLS_REPORT_YEAR_MONTH]
      ,[SALES_DT]
      ,[SLS_CENSUS_TRACT]
      ,[SLS_CONTROL_NBR]
      ,[SLS_DEALER_NAME]
      ,[SLS_TOWN_NAME]
      ,[SLS_STATE_ABBRV]
      ,[NVI_CONTROL_NBR]
      ,[NVI_DEALER_NAME]
      ,[NVI_TOWN_NAME]
      ,[NVI_STATE_ABBRV]
      ,[MAKE_DESC]
      ,[MODEL_DESC]
      ,[SERIES_TEXT]
      ,[SEGMENT_DESC]
      ,[SLS_VEHICLE_COUNT]
    FROM [SPGM_Live].[SPGM_Weekly_INV_NVI_SLS_20251107]
"""

# Execute the query and fetch all rows
full_df_live = pd.read_sql(live_query, engine)

# Display the shape and first few rows of the DataFrame
print(f"Total rows fetched: {len(full_df_live):,}")
print(full_df_live.head())

Total rows fetched: 30,625,458
  REPORT_YEAR_WEEK INV_STATE_ABBRV INV_TOWN_NAME  DAYS_LSTD_BFR_SLD  \
0          2025-42              CA    SANTA ROSA             -999.0   
1          2025-36              SC         GREER             -999.0   
2          2025-38              TX        DALLAS               62.0   
3          2025-41              TX         NIXON             -999.0   
4          2025-37              MT      MISSOULA             -999.0   

  SRC_VEH_FIRST_SCRAPED_DT TRIM_DESCRIPTION NVI_OWNSHP_DT  NVI_EFCTV_START_DT  \
0               2025-08-05               SE    1899-12-30 1899-12-30 00:00:00   
1               2024-11-14   1500 ELEVATION    1899-12-30 1899-12-30 00:00:00   
2               2025-07-30         350 BASE    2025-10-08 2025-11-04 20:41:07   
3               2025-09-28       150 TREMOR    1899-12-30 1899-12-30 00:00:00   
4               2025-07-12        PREFERRED    1899-12-30 1899-12-30 00:00:00   

  NVI_CENSUS_TRACT NVI_RPT_YYYYMM  ... SLS_STATE_ABBRV 

In [16]:
# Compare SLS_STATE_ABBRV and INV_STATE_ABBRV
print("Comparison of SLS_STATE_ABBRV and INV_STATE_ABBRV")
print("=" * 80)

# Total rows
total_rows = len(full_df_live)
print(f"Total rows: {total_rows:,}")

# Check for non-null values in both columns
both_not_null = full_df_live['SLS_STATE_ABBRV'].notna() & full_df_live['INV_STATE_ABBRV'].notna()
both_not_null_count = both_not_null.sum()

print(f"\nRows with both values non-null: {both_not_null_count:,} ({100*both_not_null_count/total_rows:.2f}%)")

# Among rows where both are not null, check how many match
if both_not_null_count > 0:
    matching = (full_df_live.loc[both_not_null, 'SLS_STATE_ABBRV'] == 
                full_df_live.loc[both_not_null, 'INV_STATE_ABBRV'])
    matching_count = matching.sum()
    
    print(f"\nAmong rows with both values:")
    print(f"  Matching states: {matching_count:,} ({100*matching_count/both_not_null_count:.2f}%)")
    print(f"  Different states: {both_not_null_count - matching_count:,} ({100*(both_not_null_count - matching_count)/both_not_null_count:.2f}%)")
    
    # Overall percentage (of all rows)
    print(f"\nOverall (of all {total_rows:,} rows):")
    print(f"  Matching states: {matching_count:,} ({100*matching_count/total_rows:.2f}%)")

# Check null patterns
sls_null = full_df_live['SLS_STATE_ABBRV'].isna().sum()
inv_null = full_df_live['INV_STATE_ABBRV'].isna().sum()

print(f"\nNull value counts:")
print(f"  SLS_STATE_ABBRV is null: {sls_null:,} ({100*sls_null/total_rows:.2f}%)")
print(f"  INV_STATE_ABBRV is null: {inv_null:,} ({100*inv_null/total_rows:.2f}%)")

# Show some examples of mismatches if they exist
if both_not_null_count > 0 and matching_count < both_not_null_count:
    print(f"\n{'-' * 80}")
    print("Sample of mismatched states (first 10):")
    mismatches = full_df_live[both_not_null & ~matching][['SLS_STATE_ABBRV', 'INV_STATE_ABBRV', 'SLS_DEALER_NAME', 'NVI_DEALER_NAME']].head(10)
    print(mismatches.to_string(index=False))

Comparison of SLS_STATE_ABBRV and INV_STATE_ABBRV
Total rows: 30,625,458

Rows with both values non-null: 30,625,458 (100.00%)

Rows with both values non-null: 30,625,458 (100.00%)

Among rows with both values:
  Matching states: 1,862,336 (6.08%)
  Different states: 28,763,122 (93.92%)

Overall (of all 30,625,458 rows):
  Matching states: 1,862,336 (6.08%)

Among rows with both values:
  Matching states: 1,862,336 (6.08%)
  Different states: 28,763,122 (93.92%)

Overall (of all 30,625,458 rows):
  Matching states: 1,862,336 (6.08%)

Null value counts:
  SLS_STATE_ABBRV is null: 0 (0.00%)
  INV_STATE_ABBRV is null: 0 (0.00%)

--------------------------------------------------------------------------------
Sample of mismatched states (first 10):

Null value counts:
  SLS_STATE_ABBRV is null: 0 (0.00%)
  INV_STATE_ABBRV is null: 0 (0.00%)

--------------------------------------------------------------------------------
Sample of mismatched states (first 10):
SLS_STATE_ABBRV INV_STATE_ABB

In [18]:
# Quick diagnostic on INV_STATE_ABBRV values
print("INV_STATE_ABBRV Value Check:")
print("=" * 80)
print(f"Total rows: {len(full_df_live):,}")
print(f"Null values: {full_df_live['INV_STATE_ABBRV'].isna().sum():,}")
print(f"Empty string values: {(full_df_live['INV_STATE_ABBRV'] == '').sum():,}")
print(f"Non-null and non-empty: {((full_df_live['INV_STATE_ABBRV'].notna()) & (full_df_live['INV_STATE_ABBRV'] != '')).sum():,}")
print("\nTop 10 most common values:")
print(full_df_live['INV_STATE_ABBRV'].value_counts().head(10))

INV_STATE_ABBRV Value Check:
Total rows: 30,625,458
Null values: 0
Empty string values: 1,640,750
Non-null and non-empty: 28,984,708

Top 10 most common values:
INV_STATE_ABBRV
CA    2937419
FL    2765416
TX    2750795
      1640750
NY    1439994
PA    1343284
NC    1305204
MI    1206603
OH    1171351
VA     918279
Name: count, dtype: int64


In [20]:
def query_make_state_sample(make, state, engine):
    """
    Query data for a specific Make and State combination for sanity checking.
    
    Parameters:
    -----------
    make : str
        The MAKE_DESC value to filter (e.g., 'FORD', 'CHEVROLET')
    state : str
        The INV_STATE_ABBRV value to filter (e.g., 'TX', 'CA')
    engine : sqlalchemy.engine
        Database engine connection
        
    Returns:
    --------
    pandas.DataFrame
        Filtered data with sales coverage analysis
    """
    query = f"""
        SELECT [REPORT_YEAR_WEEK]
          ,[INV_STATE_ABBRV]
          ,[DAYS_LSTD_BFR_SLD]
          ,[SRC_VEH_FIRST_SCRAPED_DT]
          ,[TRIM_DESCRIPTION]
          ,[NVI_OWNSHP_DT]
          ,[NVI_EFCTV_START_DT]
          ,[NVI_CENSUS_TRACT]
          ,[NVI_RPT_YYYYMM]
          ,[SLS_REPORT_YEAR_MONTH]
          ,[SALES_DT]
          ,[SLS_CENSUS_TRACT]
          ,[SLS_CONTROL_NBR]
          ,[SLS_DEALER_NAME]
          ,[SLS_TOWN_NAME]
          ,[SLS_STATE_ABBRV]
          ,[NVI_CONTROL_NBR]
          ,[NVI_DEALER_NAME]
          ,[NVI_TOWN_NAME]
          ,[NVI_STATE_ABBRV]
          ,[MAKE_DESC]
          ,[MODEL_DESC]
          ,[SERIES_TEXT]
          ,[SEGMENT_DESC]
          ,[SLS_VEHICLE_COUNT]
        FROM [SPGM_Live].[SPGM_Weekly_INV_NVI_SLS_20251107]
        WHERE [MAKE_DESC] = '{make}'
          AND [INV_STATE_ABBRV] = '{state}'
          AND [INV_STATE_ABBRV] != ''
    """
    
    print(f"Querying data for: {make} in {state}")
    print("=" * 80)
    
    df = pd.read_sql(query, engine)
    
    # Convert SALES_DT to datetime
    df['SALES_DT'] = pd.to_datetime(df['SALES_DT'], errors='coerce')
    cutoff_date = pd.Timestamp('1950-01-01')
    
    # Calculate sales coverage
    df['has_sales_coverage'] = (df['SALES_DT'].notna() & (df['SALES_DT'] >= cutoff_date)).astype(int)
    
    # Print summary
    total = len(df)
    with_coverage = df['has_sales_coverage'].sum()
    coverage_pct = (with_coverage / total * 100) if total > 0 else 0
    
    print(f"\nResults:")
    print(f"  Total rows: {total:,}")
    print(f"  Rows with sales coverage: {with_coverage:,}")
    print(f"  Coverage percentage: {coverage_pct:.1f}%")
    print(f"\nSALES_DT breakdown:")
    print(f"  Null SALES_DT: {df['SALES_DT'].isna().sum():,}")
    print(f"  Valid SALES_DT (>= 1950): {((df['SALES_DT'].notna()) & (df['SALES_DT'] >= cutoff_date)).sum():,}")
    print(f"  Invalid SALES_DT (< 1950): {((df['SALES_DT'].notna()) & (df['SALES_DT'] < cutoff_date)).sum():,}")
    
    return df

# Example usage:
# sample_df = query_make_state_sample('FORD', 'TX', engine)
# print(sample_df[['MAKE_DESC', 'INV_STATE_ABBRV', 'SALES_DT', 'has_sales_coverage']].head(20))

In [21]:
sample_df = query_make_state_sample('LINCOLN', 'AK', engine)

Querying data for: LINCOLN in AK

Results:
  Total rows: 135
  Rows with sales coverage: 0
  Coverage percentage: 0.0%

SALES_DT breakdown:
  Null SALES_DT: 0
  Valid SALES_DT (>= 1950): 0
  Invalid SALES_DT (< 1950): 135

Results:
  Total rows: 135
  Rows with sales coverage: 0
  Coverage percentage: 0.0%

SALES_DT breakdown:
  Null SALES_DT: 0
  Valid SALES_DT (>= 1950): 0
  Invalid SALES_DT (< 1950): 135


In [23]:
# View the sample data
print(f"\nViewing first 20 rows of {len(sample_df):,} total rows:")
print("=" * 80)
sample_df[['MAKE_DESC', 'INV_STATE_ABBRV', 'SALES_DT', 'SLS_STATE_ABBRV', 'has_sales_coverage']].head(135)


Viewing first 20 rows of 135 total rows:


,MAKE_DESC,INV_STATE_ABBRV,SALES_DT,SLS_STATE_ABBRV,has_sales_coverage
0,LINCOLN,AK,1899-12-30,,0
1,LINCOLN,AK,1899-12-30,,0
2,LINCOLN,AK,1899-12-30,,0
3,LINCOLN,AK,1899-12-30,,0
4,LINCOLN,AK,1899-12-30,,0
...,...,...,...,...,...
130,LINCOLN,AK,1899-12-30,,0
131,LINCOLN,AK,1899-12-30,,0
132,LINCOLN,AK,1899-12-30,,0
133,LINCOLN,AK,1899-12-30,,0


In [29]:
# Sales Coverage Analysis by Make and State (using INV_STATE_ABBRV)
print("Preparing Sales Coverage Analysis...")
print("=" * 80)

# Define valid sales date cutoff
cutoff_date = pd.Timestamp('1950-01-01')

# Convert SALES_DT to datetime
full_df_live['SALES_DT'] = pd.to_datetime(full_df_live['SALES_DT'], errors='coerce')

# Calculate skipped rows
total_rows = len(full_df_live)
null_state = full_df_live['INV_STATE_ABBRV'].isna().sum()
null_make = full_df_live['MAKE_DESC'].isna().sum()
empty_state = (full_df_live['INV_STATE_ABBRV'] == '').sum()
empty_make = (full_df_live['MAKE_DESC'] == '').sum()
both_null = (full_df_live['INV_STATE_ABBRV'].isna() & full_df_live['MAKE_DESC'].isna()).sum()

print(f"\nData Summary:")
print(f"  Total rows: {total_rows:,}")
print(f"  Rows with null INV_STATE_ABBRV: {null_state:,} ({100*null_state/total_rows:.2f}%)")
print(f"  Rows with empty string INV_STATE_ABBRV: {empty_state:,} ({100*empty_state/total_rows:.2f}%)")
print(f"  Rows with null MAKE_DESC: {null_make:,} ({100*null_make/total_rows:.2f}%)")
print(f"  Rows with empty string MAKE_DESC: {empty_make:,} ({100*empty_make/total_rows:.2f}%)")
print(f"  Rows with both null: {both_null:,} ({100*both_null/total_rows:.2f}%)")

# Filter to valid rows (both MAKE_DESC and INV_STATE_ABBRV are not null AND not empty strings)
valid_df = full_df_live[
    (full_df_live['MAKE_DESC'].notna()) & 
    (full_df_live['MAKE_DESC'] != '') &
    (full_df_live['INV_STATE_ABBRV'].notna()) & 
    (full_df_live['INV_STATE_ABBRV'] != '')
].copy()
valid_rows = len(valid_df)
skipped_rows = total_rows - valid_rows

print(f"  Rows used in analysis: {valid_rows:,} ({100*valid_rows/total_rows:.2f}%)")
print(f"  Rows skipped: {skipped_rows:,} ({100*skipped_rows/total_rows:.2f}%)")

# Create sales coverage flag
valid_df['has_sales_coverage'] = (valid_df['SALES_DT'].notna() & 
                                   (valid_df['SALES_DT'] >= cutoff_date)).astype(int)

# Diagnostic: Check overall distribution
total_with_coverage = valid_df['has_sales_coverage'].sum()
print(f"\nOverall Coverage:")
print(f"  Rows with sales coverage: {total_with_coverage:,} ({100*total_with_coverage/valid_rows:.2f}%)")
print(f"  Rows without sales coverage: {valid_rows - total_with_coverage:,} ({100*(valid_rows - total_with_coverage)/valid_rows:.2f}%)")

# Check some sample coverage percentages by Make
print(f"\nSample coverage by Make (top 5):")
make_coverage = valid_df.groupby('MAKE_DESC')['has_sales_coverage'].agg(['count', 'sum', 'mean'])
make_coverage['percentage'] = make_coverage['mean'] * 100
print(make_coverage.nlargest(5, 'count')[['count', 'sum', 'percentage']])

print(f"\nCalculating coverage matrix...")

# Create pivot table for coverage analysis
# Group by MAKE_DESC and INV_STATE_ABBRV, calculate total rows and coverage rows
coverage_data = valid_df.groupby(['MAKE_DESC', 'INV_STATE_ABBRV']).agg(
    total_rows=('has_sales_coverage', 'count'),
    coverage_rows=('has_sales_coverage', 'sum')
).reset_index()

# Calculate percentage
coverage_data['coverage_percentage'] = (coverage_data['coverage_rows'] / coverage_data['total_rows']) * 100

# Analyze coverage percentage distribution
print(f"\nCoverage Percentage Distribution:")
print(f"=" * 80)
total_combinations = len(coverage_data)
count_0_pct = (coverage_data['coverage_percentage'] == 0).sum()
count_100_pct = (coverage_data['coverage_percentage'] == 100).sum()
count_between = ((coverage_data['coverage_percentage'] > 0) & (coverage_data['coverage_percentage'] < 100)).sum()

print(f"Total Make × State combinations: {total_combinations:,}")
print(f"  0% coverage: {count_0_pct:,} combinations ({100*count_0_pct/total_combinations:.2f}%)")
print(f"  100% coverage: {count_100_pct:,} combinations ({100*count_100_pct/total_combinations:.2f}%)")
print(f"  Between 0-100%: {count_between:,} combinations ({100*count_between/total_combinations:.2f}%)")

# Show some examples of partial coverage
if count_between > 0:
    print(f"\nExamples of partial coverage (0% < coverage < 100%):")
    partial_coverage = coverage_data[(coverage_data['coverage_percentage'] > 0) & 
                                     (coverage_data['coverage_percentage'] < 100)].copy()
    partial_coverage_sorted = partial_coverage.sort_values('total_rows', ascending=False).head(10)
    print(partial_coverage_sorted[['MAKE_DESC', 'INV_STATE_ABBRV', 'total_rows', 'coverage_rows', 'coverage_percentage']].to_string(index=False))

# Identify top 20 combinations by total occurrence
top_20_combinations = coverage_data.nlargest(20, 'total_rows')[['MAKE_DESC', 'INV_STATE_ABBRV']].copy()
top_20_set = set(zip(top_20_combinations['MAKE_DESC'], top_20_combinations['INV_STATE_ABBRV']))

# Identify top 20 highest coverage percentages (excluding 0% and only for combinations with data)
top_20_coverage = coverage_data[coverage_data['total_rows'] > 0].nlargest(20, 'coverage_percentage')[['MAKE_DESC', 'INV_STATE_ABBRV']].copy()
top_20_coverage_set = set(zip(top_20_coverage['MAKE_DESC'], top_20_coverage['INV_STATE_ABBRV']))

print(f"\nTop 20 Make × State combinations by occurrence:")
top_20_display = coverage_data.nlargest(20, 'total_rows')[['MAKE_DESC', 'INV_STATE_ABBRV', 'total_rows', 'coverage_rows', 'coverage_percentage']]
top_20_display['coverage_percentage'] = top_20_display['coverage_percentage'].round(1)
print(top_20_display.to_string(index=False))

print(f"\nTop 20 Make × State combinations by highest coverage percentage:")
top_20_coverage_display = coverage_data[coverage_data['total_rows'] > 0].nlargest(20, 'coverage_percentage')[['MAKE_DESC', 'INV_STATE_ABBRV', 'total_rows', 'coverage_rows', 'coverage_percentage']]
top_20_coverage_display['coverage_percentage'] = top_20_coverage_display['coverage_percentage'].round(1)
print(top_20_coverage_display.to_string(index=False))

# Create formatted string with count and percentage
coverage_data['cell_value'] = coverage_data.apply(
    lambda row: f"{row['coverage_percentage']:.1f}% ({int(row['coverage_rows'])}/{int(row['total_rows'])})", 
    axis=1
)

# Pivot to create matrix with formatted values
coverage_matrix_display = coverage_data.pivot(index='MAKE_DESC', 
                                               columns='INV_STATE_ABBRV', 
                                               values='cell_value')

# Also create numeric matrix for calculations
coverage_matrix_numeric = coverage_data.pivot(index='MAKE_DESC', 
                                               columns='INV_STATE_ABBRV', 
                                               values='coverage_percentage')

# Fill NaN with empty string (representing 0/0 cases - no data for that combination)
coverage_matrix_display = coverage_matrix_display.fillna('')
coverage_matrix_numeric = coverage_matrix_numeric.fillna(0)

# Calculate row totals (overall coverage per Make)
row_totals = valid_df.groupby('MAKE_DESC').agg(
    total_rows=('has_sales_coverage', 'count'),
    coverage_rows=('has_sales_coverage', 'sum')
)
row_totals['percentage'] = (row_totals['coverage_rows'] / row_totals['total_rows']) * 100
row_totals['formatted'] = row_totals.apply(
    lambda row: f"{row['percentage']:.1f}% ({int(row['coverage_rows'])}/{int(row['total_rows'])})", 
    axis=1
)

# Calculate % of total dataset for each Make
row_totals['pct_of_dataset'] = (row_totals['total_rows'] / valid_rows) * 100
row_totals['pct_formatted'] = row_totals['pct_of_dataset'].apply(lambda x: f"{x:.1f}%")

coverage_matrix_display['Row_Total'] = row_totals['formatted']
coverage_matrix_display['% of Dataset'] = row_totals['pct_formatted']
coverage_matrix_numeric['Row_Total_%'] = row_totals['percentage']

# Calculate column totals (overall coverage per State)
col_totals = valid_df.groupby('INV_STATE_ABBRV').agg(
    total_rows=('has_sales_coverage', 'count'),
    coverage_rows=('has_sales_coverage', 'sum')
)
col_totals['percentage'] = (col_totals['coverage_rows'] / col_totals['total_rows']) * 100

# Calculate % of total dataset for each State
col_totals['pct_of_dataset'] = (col_totals['total_rows'] / valid_rows) * 100

# Add column totals as a new row
col_totals_dict_display = {}
col_totals_dict_numeric = {}
for state in col_totals.index:
    col_totals_dict_display[state] = f"{col_totals.loc[state, 'percentage']:.1f}% ({int(col_totals.loc[state, 'coverage_rows'])}/{int(col_totals.loc[state, 'total_rows'])})"
    col_totals_dict_numeric[state] = col_totals.loc[state, 'percentage']

# Grand total
grand_total_cov = valid_df['has_sales_coverage'].sum()
grand_total_rows = len(valid_df)
grand_total_pct = (grand_total_cov / grand_total_rows) * 100
col_totals_dict_display['Row_Total'] = f"{grand_total_pct:.1f}% ({int(grand_total_cov)}/{int(grand_total_rows)})"
col_totals_dict_display['% of Dataset'] = "100.0%"
col_totals_dict_numeric['Row_Total_%'] = grand_total_pct

coverage_matrix_display.loc['Column_Total'] = col_totals_dict_display
coverage_matrix_numeric.loc['Column_Total_%'] = col_totals_dict_numeric

# Add % of Dataset row (showing what % of total dataset each state represents)
pct_of_dataset_dict = {}
for state in col_totals.index:
    pct_of_dataset_dict[state] = f"{col_totals.loc[state, 'pct_of_dataset']:.1f}%"
pct_of_dataset_dict['Row_Total'] = "100.0%"
pct_of_dataset_dict['% of Dataset'] = "-"

coverage_matrix_display.loc['% of Dataset'] = pct_of_dataset_dict

# Sort by Row_Total_% descending (excluding the totals rows)
sorted_makes = coverage_matrix_numeric.iloc[:-1].sort_values('Row_Total_%', ascending=False).index
coverage_matrix_display_sorted = coverage_matrix_display.loc[sorted_makes.tolist() + ['Column_Total', '% of Dataset']]

print(f"\n✓ Matrix created: {len(coverage_matrix_display_sorted)-2} Makes × {len(coverage_matrix_display_sorted.columns)-2} States")
print(f"  + Row showing % of dataset per State")
print(f"  + Column showing % of dataset per Make")

# Export to Excel with formatting
output_file = 'sales_coverage_by_make_and_state.xlsx'

from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Main coverage matrix with formatted values
    coverage_matrix_display_sorted.to_excel(writer, sheet_name='Coverage_Matrix', index=True)
    
    # Summary statistics
    summary_data = {
        'Metric': [
            'Total Rows in Dataset',
            'Rows Used in Analysis',
            'Rows Skipped (null/empty Make or State)',
            'Percentage of Rows Skipped',
            'Rows with null INV_STATE_ABBRV',
            'Rows with empty INV_STATE_ABBRV',
            'Rows with null MAKE_DESC',
            'Rows with empty MAKE_DESC',
            'Overall Sales Coverage %',
            'Number of Makes',
            'Number of States',
            'Total Make × State Combinations',
            'Combinations with 0% Coverage',
            'Combinations with 100% Coverage',
            'Combinations with Partial Coverage',
            'Highlighted - Gold Background',
            'Highlighted - Green Text',
            'Highlighted - Both',
        ],
        'Value': [
            f"{total_rows:,}",
            f"{valid_rows:,}",
            f"{skipped_rows:,}",
            f"{100*skipped_rows/total_rows:.2f}%",
            f"{null_state:,}",
            f"{empty_state:,}",
            f"{null_make:,}",
            f"{empty_make:,}",
            f"{grand_total_pct:.2f}%",
            len(coverage_matrix_display_sorted) - 2,  # Exclude both totals rows
            len(coverage_matrix_display_sorted.columns) - 2,  # Exclude both totals columns
            f"{total_combinations:,}",
            f"{count_0_pct:,} ({100*count_0_pct/total_combinations:.2f}%)",
            f"{count_100_pct:,} ({100*count_100_pct/total_combinations:.2f}%)",
            f"{count_between:,} ({100*count_between/total_combinations:.2f}%)",
            'Top 20 Make × State by occurrence',
            'Top 20 Make × State by highest coverage %',
            'Cells in both top 20 lists'
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    # Format the Coverage_Matrix sheet
    worksheet = writer.sheets['Coverage_Matrix']
    
    # Set column widths
    worksheet.column_dimensions['A'].width = 20  # MAKE_DESC column
    for col in range(2, len(coverage_matrix_display_sorted.columns) + 2):
        col_letter = get_column_letter(col)
        worksheet.column_dimensions[col_letter].width = 18  # State columns
    
    # Set row height for better readability
    for row in range(1, len(coverage_matrix_display_sorted) + 2):
        worksheet.row_dimensions[row].height = 20
    
    # Format header row
    header_font = Font(bold=True, size=11)
    header_fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    header_alignment = Alignment(horizontal='center', vertical='center')
    
    for cell in worksheet[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    
    # Format MAKE_DESC column (first column)
    for row in range(2, len(coverage_matrix_display_sorted) + 2):
        cell = worksheet.cell(row=row, column=1)
        cell.font = Font(bold=True, size=10)
        cell.alignment = Alignment(horizontal='left', vertical='center')
    
    # Center align all data cells
    for row in range(2, len(coverage_matrix_display_sorted) + 2):
        for col in range(2, len(coverage_matrix_display_sorted.columns) + 2):
            cell = worksheet.cell(row=row, column=col)
            cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Highlight top 20 combinations by occurrence (gold background) and highest coverage % (green text)
    highlight_fill_gold = PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid')  # Gold background
    font_green = Font(bold=True, size=10, color='00CC00')  # Bright green text, bold for emphasis
    font_black = Font(bold=True, size=10, color='000000')  # Black text
    
    # Create mapping of column names to column indices
    state_columns = [col for col in coverage_matrix_display_sorted.columns if col not in ['Row_Total', '% of Dataset']]
    
    # Apply highlighting based on which top 20 list(s) each cell is in
    for make in coverage_matrix_display_sorted.index:
        if make in ['Column_Total', '% of Dataset']:
            continue  # Skip totals rows
            
        for state in state_columns:
            combination = (make, state)
            in_top_occurrence = combination in top_20_set
            in_top_coverage = combination in top_20_coverage_set
            
            if in_top_occurrence or in_top_coverage:
                # Find the row and column indices
                row_idx = list(coverage_matrix_display_sorted.index).index(make) + 2
                col_idx = state_columns.index(state) + 2
                cell = worksheet.cell(row=row_idx, column=col_idx)
                
                # Apply styling based on which list(s) the combination is in
                if in_top_occurrence and in_top_coverage:
                    # Both: Gold background + Green text
                    cell.fill = highlight_fill_gold
                    cell.font = font_green
                elif in_top_occurrence:
                    # Only top occurrence: Gold background + Black text
                    cell.fill = highlight_fill_gold
                    cell.font = font_black
                else:
                    # Only top coverage: Green text (no background fill)
                    cell.font = font_green
    
    # Format Column_Total row (second to last row)
    column_total_row = len(coverage_matrix_display_sorted)
    totals_fill = PatternFill(start_color='FFEB9C', end_color='FFEB9C', fill_type='solid')
    for col in range(1, len(coverage_matrix_display_sorted.columns) + 2):
        cell = worksheet.cell(row=column_total_row, column=col)
        cell.fill = totals_fill
        cell.font = Font(bold=True, size=10)
    
    # Format % of Dataset row (last row)
    pct_dataset_row = len(coverage_matrix_display_sorted) + 1
    pct_fill = PatternFill(start_color='CCE5FF', end_color='CCE5FF', fill_type='solid')  # Light blue
    for col in range(1, len(coverage_matrix_display_sorted.columns) + 2):
        cell = worksheet.cell(row=pct_dataset_row, column=col)
        cell.fill = pct_fill
        cell.font = Font(bold=True, size=10)
    
    # Format % of Dataset column (last column before the formatting)
    pct_dataset_col = len(coverage_matrix_display_sorted.columns) + 1
    for row in range(2, len(coverage_matrix_display_sorted) + 2):
        cell = worksheet.cell(row=row, column=pct_dataset_col)
        cell.fill = pct_fill
        cell.font = Font(bold=True, size=10)
    
    # Format Summary sheet
    summary_ws = writer.sheets['Summary']
    summary_ws.column_dimensions['A'].width = 40
    summary_ws.column_dimensions['B'].width = 25
    
    # Format header
    for cell in summary_ws[1]:
        cell.font = Font(bold=True, size=11)
        cell.fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
        cell.alignment = Alignment(horizontal='center', vertical='center')

print(f"\n✓ Results exported to: {output_file}")
print(f"  - Sheet 1: Coverage_Matrix (percentage + counts, formatted)")
print(f"  - Sheet 2: Summary (statistics + coverage distribution)")
print("\nFormatting applied:")
print(f"  ✓ Column width: 20 for Makes, 18 for States")
print(f"  ✓ Row height: 20 for better readability")
print(f"  ✓ Headers: Bold with gray background")
print(f"  ✓ Column_Total row: Bold with yellow background")
print(f"  ✓ % of Dataset row/column: Bold with light blue background")
print(f"  ✓ Top 20 by occurrence: Gold background")
print(f"  ✓ Top 20 by highest coverage %: Green text")
print(f"  ✓ In both top 20 lists: Gold background + Green text")
print(f"  ✓ Center-aligned data cells")
print("\nAnalysis complete!")


Preparing Sales Coverage Analysis...

Data Summary:
  Total rows: 30,625,458
  Rows with null INV_STATE_ABBRV: 0 (0.00%)
  Rows with empty string INV_STATE_ABBRV: 1,640,750 (5.36%)
  Rows with null MAKE_DESC: 0 (0.00%)
  Rows with empty string MAKE_DESC: 0 (0.00%)
  Rows with both null: 0 (0.00%)
  Rows used in analysis: 28,984,708 (94.64%)
  Rows skipped: 1,640,750 (5.36%)

Overall Coverage:
  Rows with sales coverage: 271,008 (0.94%)
  Rows without sales coverage: 28,713,700 (99.06%)

Sample coverage by Make (top 5):
             count    sum  percentage
MAKE_DESC                            
FORD       5522526  44868    0.812454
CHEVROLET  3297781  30530    0.925774
TOYOTA     2539017  20512    0.807872
HYUNDAI    2025595  17549    0.866363
HONDA      1959606  19118    0.975604

Calculating coverage matrix...

Coverage Percentage Distribution:
Total Make × State combinations: 1,692
  0% coverage: 560 combinations (33.10%)
  100% coverage: 0 combinations (0.00%)
  Between 0-100%: 1,13

In [ ]:
#Some experiments to verify things. First: State

In [5]:
# Get all distinct INV_STATE_ABBRV values
print("Distinct INV_STATE_ABBRV Values")
print("=" * 80)

# Get unique values (including null/NaN)
unique_states = full_df_live['INV_STATE_ABBRV'].unique()

# Count statistics
total_rows = len(full_df_live)
null_count = full_df_live['INV_STATE_ABBRV'].isna().sum()
empty_count = (full_df_live['INV_STATE_ABBRV'] == '').sum()
non_null_count = full_df_live['INV_STATE_ABBRV'].notna().sum()

print(f"\nTotal rows in dataframe: {total_rows:,}")
print(f"Null/NaN values: {null_count:,} ({100*null_count/total_rows:.2f}%)")
print(f"Empty string values: {empty_count:,} ({100*empty_count/total_rows:.2f}%)")
print(f"Non-null values: {non_null_count:,} ({100*non_null_count/total_rows:.2f}%)")
print(f"\nNumber of distinct values (including null): {len(unique_states)}")

# Get value counts sorted by frequency
print("\n" + "=" * 80)
print("All Distinct Values (sorted by frequency):")
print("=" * 80)
value_counts = full_df_live['INV_STATE_ABBRV'].value_counts(dropna=False)
print(f"\n{'State':<15} {'Count':<15} {'Percentage'}")
print("-" * 50)
for state, count in value_counts.items():
    pct = 100 * count / total_rows
    state_display = str(state) if pd.notna(state) else "NaN/Null"
    print(f"{state_display:<15} {count:<15,} {pct:>6.2f}%")

print("\n" + "=" * 80)
print(f"Alphabetically sorted list of non-null states:")
print("=" * 80)
non_null_states = sorted([s for s in unique_states if pd.notna(s) and s != ''])
print(non_null_states)

Distinct INV_STATE_ABBRV Values

Total rows in dataframe: 30,625,458
Null/NaN values: 0 (0.00%)
Empty string values: 1,640,750 (5.36%)
Non-null values: 30,625,458 (100.00%)

Number of distinct values (including null): 52

All Distinct Values (sorted by frequency):

State           Count           Percentage
--------------------------------------------------
CA              2,937,419         9.59%
FL              2,765,416         9.03%
TX              2,750,795         8.98%
                1,640,750         5.36%
NY              1,439,994         4.70%
PA              1,343,284         4.39%
NC              1,305,204         4.26%
MI              1,206,603         3.94%
OH              1,171,351         3.82%
VA              918,279           3.00%
IL              916,515           2.99%
GA              896,582           2.93%
NJ              653,307           2.13%
MA              608,698           1.99%
MD              585,000           1.91%
AZ              571,490           1.87%


In [6]:
len(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'])

51

In [7]:
# Compare SLS_STATE_ABBRV and INV_STATE_ABBRV where both are present
print("Comparison of SLS_STATE_ABBRV and INV_STATE_ABBRV")
print("=" * 80)

total_rows = len(full_df_live)
print(f"Total rows in dataframe: {total_rows:,}")

# Check for null/empty values in each column
sls_null = full_df_live['SLS_STATE_ABBRV'].isna().sum()
sls_empty = (full_df_live['SLS_STATE_ABBRV'] == '').sum()
inv_null = full_df_live['INV_STATE_ABBRV'].isna().sum()
inv_empty = (full_df_live['INV_STATE_ABBRV'] == '').sum()

print(f"\nSLS_STATE_ABBRV:")
print(f"  Null/NaN: {sls_null:,} ({100*sls_null/total_rows:.2f}%)")
print(f"  Empty string: {sls_empty:,} ({100*sls_empty/total_rows:.2f}%)")
print(f"  Non-null and non-empty: {total_rows - sls_null - sls_empty:,} ({100*(total_rows - sls_null - sls_empty)/total_rows:.2f}%)")

print(f"\nINV_STATE_ABBRV:")
print(f"  Null/NaN: {inv_null:,} ({100*inv_null/total_rows:.2f}%)")
print(f"  Empty string: {inv_empty:,} ({100*inv_empty/total_rows:.2f}%)")
print(f"  Non-null and non-empty: {total_rows - inv_null - inv_empty:,} ({100*(total_rows - inv_null - inv_empty)/total_rows:.2f}%)")

# Filter to rows where BOTH columns have non-null and non-empty values
both_present = (
    full_df_live['SLS_STATE_ABBRV'].notna() & 
    (full_df_live['SLS_STATE_ABBRV'] != '') &
    full_df_live['INV_STATE_ABBRV'].notna() & 
    (full_df_live['INV_STATE_ABBRV'] != '')
)
both_present_count = both_present.sum()

print("\n" + "=" * 80)
print("Analysis of rows where BOTH columns have values:")
print("=" * 80)
print(f"Rows with both SLS_STATE_ABBRV and INV_STATE_ABBRV present: {both_present_count:,} ({100*both_present_count/total_rows:.2f}%)")

if both_present_count > 0:
    # Among rows where both are present, check how many match
    matching = (
        full_df_live.loc[both_present, 'SLS_STATE_ABBRV'] == 
        full_df_live.loc[both_present, 'INV_STATE_ABBRV']
    )
    matching_count = matching.sum()
    different_count = both_present_count - matching_count
    
    print(f"\nAmong the {both_present_count:,} rows with both values:")
    print(f"  Matching states: {matching_count:,} ({100*matching_count/both_present_count:.2f}%)")
    print(f"  Different states: {different_count:,} ({100*different_count/both_present_count:.2f}%)")
    
    # Show some examples of mismatches if they exist
    if different_count > 0:
        print("\n" + "=" * 80)
        print(f"Sample of rows with DIFFERENT states (first 20):")
        print("=" * 80)
        mismatches = full_df_live[both_present & ~matching][
            ['SLS_STATE_ABBRV', 'INV_STATE_ABBRV', 'SLS_DEALER_NAME', 
             'NVI_DEALER_NAME', 'MAKE_DESC', 'MODEL_DESC']
        ].head(20)
        print(mismatches.to_string(index=False))
        
        # Show distribution of mismatches by state pair
        print("\n" + "=" * 80)
        print("Most common state mismatches (Top 10):")
        print("=" * 80)
        mismatch_pairs = full_df_live[both_present & ~matching].groupby(
            ['SLS_STATE_ABBRV', 'INV_STATE_ABBRV']
        ).size().reset_index(name='count')
        mismatch_pairs_sorted = mismatch_pairs.sort_values('count', ascending=False).head(10)
        print(f"\n{'SLS State':<12} {'INV State':<12} {'Count':<12} {'% of Mismatches'}")
        print("-" * 60)
        for _, row in mismatch_pairs_sorted.iterrows():
            pct = 100 * row['count'] / different_count
            print(f"{row['SLS_STATE_ABBRV']:<12} {row['INV_STATE_ABBRV']:<12} {row['count']:<12,} {pct:>6.2f}%")
    else:
        print("\n✓ All rows with both values present have MATCHING states!")
else:
    print("\n⚠ No rows found where both SLS_STATE_ABBRV and INV_STATE_ABBRV are present!")

Comparison of SLS_STATE_ABBRV and INV_STATE_ABBRV
Total rows in dataframe: 30,625,458

SLS_STATE_ABBRV:
  Null/NaN: 0 (0.00%)
  Empty string: 30,344,808 (99.08%)
  Non-null and non-empty: 280,650 (0.92%)

INV_STATE_ABBRV:
  Null/NaN: 0 (0.00%)
  Empty string: 1,640,750 (5.36%)
  Non-null and non-empty: 28,984,708 (94.64%)

Analysis of rows where BOTH columns have values:
Rows with both SLS_STATE_ABBRV and INV_STATE_ABBRV present: 265,885 (0.87%)

Among the 265,885 rows with both values:
  Matching states: 236,351 (88.89%)
  Different states: 29,534 (11.11%)

Sample of rows with DIFFERENT states (first 20):
SLS_STATE_ABBRV INV_STATE_ABBRV                          SLS_DEALER_NAME NVI_DEALER_NAME MAKE_DESC     MODEL_DESC
             OH              MI                          JIM WHITE HONDA                     HONDA       PASSPORT
             NV              CA                              GAUDIN FORD                      FORD MUSTANG MACH-E
             VA              NC        RICK 

In [10]:
def check_sales_coverage_by_state(df, state):
    """
    Check sales coverage for a specific state across all makes.
    
    Sales coverage is defined as:
    - SALES_DT >= 1950-01-01 (not null and not before 1950)
    - AND SLS_STATE_ABBRV is present (not null and not empty string)
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe to analyze (e.g., full_df_live)
    state : str
        The INV_STATE_ABBRV value to filter (e.g., 'TX', 'CA')
        
    Returns:
    --------
    dict with keys:
        - state: The state analyzed
        - total_rows: Total number of rows for this state
        - coverage_count: Number of rows with sales coverage
        - coverage_percentage: Percentage of rows with coverage
        - sales_dt_valid: Number of rows with valid SALES_DT
        - sls_state_present: Number of rows with SLS_STATE_ABBRV present
        - filtered_df: DataFrame filtered to this state
    """
    # Filter to the specified state (using INV_STATE_ABBRV)
    state_df = df[
        (df['INV_STATE_ABBRV'] == state) & 
        (df['INV_STATE_ABBRV'].notna()) & 
        (df['INV_STATE_ABBRV'] != '')
    ].copy()
    
    total = len(state_df)
    
    if total == 0:
        print(f"\n⚠ No data found for state: {state}")
        return {
            'state': state,
            'total_rows': 0,
            'coverage_count': 0,
            'coverage_percentage': 0.0,
            'sales_dt_valid': 0,
            'sls_state_present': 0,
            'filtered_df': state_df
        }
    
    # Convert SALES_DT to datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(state_df['SALES_DT']):
        state_df['SALES_DT'] = pd.to_datetime(state_df['SALES_DT'], errors='coerce')
    
    cutoff_date = pd.Timestamp('1950-01-01')
    
    # Check SALES_DT status
    sales_dt_null = state_df['SALES_DT'].isna().sum()
    sales_dt_before_1950 = ((state_df['SALES_DT'].notna()) & (state_df['SALES_DT'] < cutoff_date)).sum()
    sales_dt_valid = ((state_df['SALES_DT'].notna()) & (state_df['SALES_DT'] >= cutoff_date)).sum()
    
    # Check SLS_STATE_ABBRV status
    sls_state_null = state_df['SLS_STATE_ABBRV'].isna().sum()
    sls_state_empty = (state_df['SLS_STATE_ABBRV'] == '').sum()
    sls_state_present = ((state_df['SLS_STATE_ABBRV'].notna()) & (state_df['SLS_STATE_ABBRV'] != '')).sum()
    
    # Calculate coverage: BOTH conditions must be true
    has_coverage = (
        (state_df['SALES_DT'].notna()) & 
        (state_df['SALES_DT'] >= cutoff_date) &
        (state_df['SLS_STATE_ABBRV'].notna()) & 
        (state_df['SLS_STATE_ABBRV'] != '')
    )
    coverage_count = has_coverage.sum()
    coverage_pct = (coverage_count / total * 100) if total > 0 else 0
    
    # Calculate non-coverage breakdown
    no_coverage = ~has_coverage
    no_coverage_count = no_coverage.sum()
    
    # Among non-coverage rows, break down by reason
    missing_both = (
        (state_df['SALES_DT'].isna() | (state_df['SALES_DT'] < cutoff_date)) &
        (state_df['SLS_STATE_ABBRV'].isna() | (state_df['SLS_STATE_ABBRV'] == ''))
    ).sum()
    
    missing_only_sales_dt = (
        (state_df['SALES_DT'].isna() | (state_df['SALES_DT'] < cutoff_date)) &
        (state_df['SLS_STATE_ABBRV'].notna() & (state_df['SLS_STATE_ABBRV'] != ''))
    ).sum()
    
    missing_only_sls_state = (
        (state_df['SALES_DT'].notna() & (state_df['SALES_DT'] >= cutoff_date)) &
        (state_df['SLS_STATE_ABBRV'].isna() | (state_df['SLS_STATE_ABBRV'] == ''))
    ).sum()
    
    # Print results
    print(f"\nSales Coverage Analysis for State: {state}")
    print("=" * 80)
    print(f"Total rows: {total:,}")
    
    print(f"\nSALES_DT Status:")
    print(f"  Null: {sales_dt_null:,} ({100*sales_dt_null/total:.1f}%)")
    print(f"  Before 1950: {sales_dt_before_1950:,} ({100*sales_dt_before_1950/total:.1f}%)")
    print(f"  Valid (>= 1950): {sales_dt_valid:,} ({100*sales_dt_valid/total:.1f}%)")
    
    print(f"\nSLS_STATE_ABBRV Status:")
    print(f"  Null: {sls_state_null:,} ({100*sls_state_null/total:.1f}%)")
    print(f"  Empty string: {sls_state_empty:,} ({100*sls_state_empty/total:.1f}%)")
    print(f"  Present: {sls_state_present:,} ({100*sls_state_present/total:.1f}%)")
    
    print(f"\n{'='*80}")
    print(f"SALES COVERAGE (SALES_DT >= 1950 AND SLS_STATE_ABBRV present):")
    print(f"  With coverage: {coverage_count:,} ({coverage_pct:.1f}%)")
    print(f"  Without coverage: {no_coverage_count:,} ({100*no_coverage_count/total:.1f}%)")
    
    if no_coverage_count > 0:
        print(f"\nBreakdown of rows WITHOUT coverage:")
        print(f"  Missing both: {missing_both:,} ({100*missing_both/no_coverage_count:.1f}% of non-coverage)")
        print(f"  Missing only SALES_DT: {missing_only_sales_dt:,} ({100*missing_only_sales_dt/no_coverage_count:.1f}% of non-coverage)")
        print(f"  Missing only SLS_STATE_ABBRV: {missing_only_sls_state:,} ({100*missing_only_sls_state/no_coverage_count:.1f}% of non-coverage)")
    
    return {
        'state': state,
        'total_rows': total,
        'coverage_count': coverage_count,
        'coverage_percentage': coverage_pct,
        'sales_dt_valid': sales_dt_valid,
        'sls_state_present': sls_state_present,
        'filtered_df': state_df
    }

# Example usage:
# result = check_sales_coverage_by_state(full_df_live, 'TX')
# result = check_sales_coverage_by_state(full_df_live, 'CA')
# Access the filtered dataframe: result['filtered_df']

In [15]:
result = check_sales_coverage_by_state(full_df_live, "KS")


Sales Coverage Analysis for State: KS
Total rows: 204,126

SALES_DT Status:
  Null: 0 (0.0%)
  Before 1950: 202,609 (99.3%)
  Valid (>= 1950): 1,517 (0.7%)

SLS_STATE_ABBRV Status:
  Null: 0 (0.0%)
  Empty string: 202,662 (99.3%)
  Present: 1,464 (0.7%)

SALES COVERAGE (SALES_DT >= 1950 AND SLS_STATE_ABBRV present):
  With coverage: 1,464 (0.7%)
  Without coverage: 202,662 (99.3%)

Breakdown of rows WITHOUT coverage:
  Missing both: 202,609 (100.0% of non-coverage)
  Missing only SALES_DT: 0 (0.0% of non-coverage)
  Missing only SLS_STATE_ABBRV: 53 (0.0% of non-coverage)


In [18]:
# Sales Coverage Analysis by Make and (State, Town) Tuple
print("Preparing Sales Coverage Analysis by Make and (State, Town)...")
print("=" * 80)

# Define valid sales date cutoff
cutoff_date = pd.Timestamp('1950-01-01')

# Ensure SALES_DT is datetime
if not pd.api.types.is_datetime64_any_dtype(full_df_live['SALES_DT']):
    full_df_live['SALES_DT'] = pd.to_datetime(full_df_live['SALES_DT'], errors='coerce')

# Calculate data quality metrics
total_rows = len(full_df_live)
null_state = full_df_live['INV_STATE_ABBRV'].isna().sum()
null_town = full_df_live['INV_TOWN_NAME'].isna().sum()
null_make = full_df_live['MAKE_DESC'].isna().sum()
empty_state = (full_df_live['INV_STATE_ABBRV'] == '').sum()
empty_town = (full_df_live['INV_TOWN_NAME'] == '').sum()
empty_make = (full_df_live['MAKE_DESC'] == '').sum()

print(f"\nData Summary:")
print(f"  Total rows: {total_rows:,}")
print(f"  Rows with null INV_STATE_ABBRV: {null_state:,} ({100*null_state/total_rows:.2f}%)")
print(f"  Rows with empty string INV_STATE_ABBRV: {empty_state:,} ({100*empty_state/total_rows:.2f}%)")
print(f"  Rows with null INV_TOWN_NAME: {null_town:,} ({100*null_town/total_rows:.2f}%)")
print(f"  Rows with empty string INV_TOWN_NAME: {empty_town:,} ({100*empty_town/total_rows:.2f}%)")
print(f"  Rows with null MAKE_DESC: {null_make:,} ({100*null_make/total_rows:.2f}%)")
print(f"  Rows with empty string MAKE_DESC: {empty_make:,} ({100*empty_make/total_rows:.2f}%)")

# Filter to valid rows (MAKE_DESC, INV_STATE_ABBRV, and INV_TOWN_NAME all not null AND not empty)
valid_df = full_df_live[
    (full_df_live['MAKE_DESC'].notna()) & 
    (full_df_live['MAKE_DESC'] != '') &
    (full_df_live['INV_STATE_ABBRV'].notna()) & 
    (full_df_live['INV_STATE_ABBRV'] != '') &
    (full_df_live['INV_TOWN_NAME'].notna()) & 
    (full_df_live['INV_TOWN_NAME'] != '')
].copy()

valid_rows = len(valid_df)
skipped_rows = total_rows - valid_rows

print(f"  Rows used in analysis: {valid_rows:,} ({100*valid_rows/total_rows:.2f}%)")
print(f"  Rows skipped: {skipped_rows:,} ({100*skipped_rows/total_rows:.2f}%)")

# Create (State, Town) tuple column
valid_df['STATE_TOWN'] = valid_df.apply(
    lambda row: f"{row['INV_STATE_ABBRV']}, {row['INV_TOWN_NAME']}", 
    axis=1
)

# Create sales coverage flag
valid_df['has_sales_coverage'] = (
    (valid_df['SALES_DT'].notna()) & 
    (valid_df['SALES_DT'] >= cutoff_date) &
    (valid_df['SLS_STATE_ABBRV'].notna()) & 
    (valid_df['SLS_STATE_ABBRV'] != '')
).astype(int)

# Overall coverage statistics
total_with_coverage = valid_df['has_sales_coverage'].sum()
print(f"\nOverall Coverage:")
print(f"  Rows with sales coverage: {total_with_coverage:,} ({100*total_with_coverage/valid_rows:.2f}%)")
print(f"  Rows without sales coverage: {valid_rows - total_with_coverage:,} ({100*(valid_rows - total_with_coverage)/valid_rows:.2f}%)")

print(f"\nCalculating coverage by Make × (State, Town)...")

# Group by MAKE_DESC and STATE_TOWN tuple
coverage_data = valid_df.groupby(['MAKE_DESC', 'STATE_TOWN', 'INV_STATE_ABBRV']).agg(
    total_rows=('has_sales_coverage', 'count'),
    coverage_rows=('has_sales_coverage', 'sum')
).reset_index()

# Calculate percentage
coverage_data['coverage_percentage'] = (coverage_data['coverage_rows'] / coverage_data['total_rows']) * 100

# Summary statistics
print(f"\nCoverage Statistics:")
print(f"=" * 80)
total_combinations = len(coverage_data)
unique_towns = coverage_data['STATE_TOWN'].nunique()
unique_states = coverage_data['INV_STATE_ABBRV'].nunique()
unique_makes = coverage_data['MAKE_DESC'].nunique()

print(f"Total Make × (State, Town) combinations: {total_combinations:,}")
print(f"Unique Makes: {unique_makes:,}")
print(f"Unique States: {unique_states:,}")
print(f"Unique (State, Town) tuples: {unique_towns:,}")

# Coverage distribution
count_0_pct = (coverage_data['coverage_percentage'] == 0).sum()
count_100_pct = (coverage_data['coverage_percentage'] == 100).sum()
count_between = ((coverage_data['coverage_percentage'] > 0) & (coverage_data['coverage_percentage'] < 100)).sum()

print(f"\nCoverage Distribution:")
print(f"  0% coverage: {count_0_pct:,} combinations ({100*count_0_pct/total_combinations:.2f}%)")
print(f"  100% coverage: {count_100_pct:,} combinations ({100*count_100_pct/total_combinations:.2f}%)")
print(f"  Between 0-100%: {count_between:,} combinations ({100*count_between/total_combinations:.2f}%)")

# Top 20 combinations by occurrence
print(f"\n{'='*80}")
print("Top 20 Make × (State, Town) combinations by occurrence:")
print(f"{'='*80}")
top_20_display = coverage_data.nlargest(20, 'total_rows')[
    ['MAKE_DESC', 'STATE_TOWN', 'total_rows', 'coverage_rows', 'coverage_percentage']
]
top_20_display['coverage_percentage'] = top_20_display['coverage_percentage'].round(1)
print(top_20_display.to_string(index=False))

# Top 20 by highest coverage percentage
print(f"\n{'='*80}")
print("Top 20 Make × (State, Town) combinations by highest coverage %:")
print(f"{'='*80}")
top_20_coverage_display = coverage_data[coverage_data['total_rows'] > 0].nlargest(20, 'coverage_percentage')[
    ['MAKE_DESC', 'STATE_TOWN', 'total_rows', 'coverage_rows', 'coverage_percentage']
]
top_20_coverage_display['coverage_percentage'] = top_20_coverage_display['coverage_percentage'].round(1)
print(top_20_coverage_display.to_string(index=False))

# Town frequency analysis
print(f"\n{'='*80}")
print("Top 20 (State, Town) tuples by total occurrence (across all makes):")
print(f"{'='*80}")
town_totals = coverage_data.groupby('STATE_TOWN').agg(
    total_rows=('total_rows', 'sum'),
    total_coverage=('coverage_rows', 'sum'),
    num_makes=('MAKE_DESC', 'nunique')
).reset_index()
town_totals['coverage_pct'] = (town_totals['total_coverage'] / town_totals['total_rows']) * 100
town_totals_sorted = town_totals.sort_values('total_rows', ascending=False).head(20)
print(f"\n{'State, Town':<40} {'Total Rows':<15} {'Makes':<10} {'Coverage %'}")
print("-" * 80)
for _, row in town_totals_sorted.iterrows():
    print(f"{row['STATE_TOWN']:<40} {int(row['total_rows']):<15,} {int(row['num_makes']):<10} {row['coverage_pct']:>6.1f}%")

print(f"\n✓ Analysis complete!")
print(f"  Total combinations: {total_combinations:,}")
print(f"  Unique (State, Town) tuples: {unique_towns:,}")
print(f"  Ready for export or further analysis")

Preparing Sales Coverage Analysis by Make and (State, Town)...

Data Summary:
  Total rows: 30,625,458
  Rows with null INV_STATE_ABBRV: 0 (0.00%)
  Rows with empty string INV_STATE_ABBRV: 1,640,750 (5.36%)
  Rows with null INV_TOWN_NAME: 0 (0.00%)
  Rows with empty string INV_TOWN_NAME: 1,640,750 (5.36%)
  Rows with null MAKE_DESC: 0 (0.00%)
  Rows with empty string MAKE_DESC: 0 (0.00%)
  Rows used in analysis: 28,984,708 (94.64%)
  Rows skipped: 1,640,750 (5.36%)

Overall Coverage:
  Rows with sales coverage: 265,885 (0.92%)
  Rows without sales coverage: 28,718,823 (99.08%)

Calculating coverage by Make × (State, Town)...

Coverage Statistics:
Total Make × (State, Town) combinations: 26,421
Unique Makes: 41
Unique States: 51
Unique (State, Town) tuples: 4,316

Coverage Distribution:
  0% coverage: 14,217 combinations (53.81%)
  100% coverage: 5 combinations (0.02%)
  Between 0-100%: 12,199 combinations (46.17%)

Top 20 Make × (State, Town) combinations by occurrence:
MAKE_DESC      

In [19]:
# Export Make × (State, Town) Coverage Analysis to Excel
print("\nExporting to Excel with formatting...")
print("=" * 80)

from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

output_file = 'sales_coverage_by_make_and_state_town.xlsx'

# Identify top 20 combinations by occurrence and coverage for highlighting
top_20_occurrence = coverage_data.nlargest(20, 'total_rows')[['MAKE_DESC', 'STATE_TOWN']].copy()
top_20_occurrence_set = set(zip(top_20_occurrence['MAKE_DESC'], top_20_occurrence['STATE_TOWN']))

top_20_coverage = coverage_data[coverage_data['total_rows'] > 0].nlargest(20, 'coverage_percentage')[['MAKE_DESC', 'STATE_TOWN']].copy()
top_20_coverage_set = set(zip(top_20_coverage['MAKE_DESC'], top_20_coverage['STATE_TOWN']))

# Create formatted display data
coverage_data_display = coverage_data.copy()
coverage_data_display['formatted'] = coverage_data_display.apply(
    lambda row: f"{row['coverage_percentage']:.1f}% ({int(row['coverage_rows'])}/{int(row['total_rows'])})", 
    axis=1
)

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Sheet 1: Top combinations by occurrence
    print("  Creating Top 20 by Occurrence sheet...")
    top_20_occur_export = coverage_data.nlargest(20, 'total_rows')[
        ['MAKE_DESC', 'INV_STATE_ABBRV', 'STATE_TOWN', 'total_rows', 'coverage_rows', 'coverage_percentage']
    ].copy()
    top_20_occur_export.columns = ['Make', 'State', 'State, Town', 'Total Rows', 'Coverage Rows', 'Coverage %']
    top_20_occur_export['Coverage %'] = top_20_occur_export['Coverage %'].round(1)
    top_20_occur_export.to_excel(writer, sheet_name='Top 20 by Occurrence', index=False)
    
    ws1 = writer.sheets['Top 20 by Occurrence']
    # Format header
    header_font = Font(bold=True, size=11)
    header_fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    for cell in ws1[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center')
    # Set column widths
    ws1.column_dimensions['A'].width = 15
    ws1.column_dimensions['B'].width = 10
    ws1.column_dimensions['C'].width = 35
    ws1.column_dimensions['D'].width = 15
    ws1.column_dimensions['E'].width = 15
    ws1.column_dimensions['F'].width = 12
    # Highlight in yellow/gold
    gold_fill = PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid')
    for row in range(2, len(top_20_occur_export) + 2):
        for col in range(1, 7):
            ws1.cell(row=row, column=col).fill = gold_fill
            ws1.cell(row=row, column=col).alignment = Alignment(horizontal='center', vertical='center')
    
    # Sheet 2: Top combinations by coverage %
    print("  Creating Top 20 by Coverage % sheet...")
    top_20_cov_export = coverage_data[coverage_data['total_rows'] > 0].nlargest(20, 'coverage_percentage')[
        ['MAKE_DESC', 'INV_STATE_ABBRV', 'STATE_TOWN', 'total_rows', 'coverage_rows', 'coverage_percentage']
    ].copy()
    top_20_cov_export.columns = ['Make', 'State', 'State, Town', 'Total Rows', 'Coverage Rows', 'Coverage %']
    top_20_cov_export['Coverage %'] = top_20_cov_export['Coverage %'].round(1)
    top_20_cov_export.to_excel(writer, sheet_name='Top 20 by Coverage', index=False)
    
    ws2 = writer.sheets['Top 20 by Coverage']
    # Format header
    for cell in ws2[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center')
    # Set column widths
    ws2.column_dimensions['A'].width = 15
    ws2.column_dimensions['B'].width = 10
    ws2.column_dimensions['C'].width = 35
    ws2.column_dimensions['D'].width = 15
    ws2.column_dimensions['E'].width = 15
    ws2.column_dimensions['F'].width = 12
    # Highlight in green
    green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
    for row in range(2, len(top_20_cov_export) + 2):
        for col in range(1, 7):
            ws2.cell(row=row, column=col).fill = green_fill
            ws2.cell(row=row, column=col).alignment = Alignment(horizontal='center', vertical='center')
    
    # Sheet 3: Top towns by occurrence
    print("  Creating Top Towns sheet...")
    top_towns_export = town_totals.sort_values('total_rows', ascending=False).head(30).copy()
    top_towns_export.columns = ['State, Town', 'Total Rows', 'Total Coverage', 'Num Makes', 'Coverage %']
    top_towns_export['Coverage %'] = top_towns_export['Coverage %'].round(1)
    top_towns_export.to_excel(writer, sheet_name='Top Towns', index=False)
    
    ws3 = writer.sheets['Top Towns']
    # Format header
    for cell in ws3[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center')
    # Set column widths
    ws3.column_dimensions['A'].width = 35
    ws3.column_dimensions['B'].width = 15
    ws3.column_dimensions['C'].width = 15
    ws3.column_dimensions['D'].width = 12
    ws3.column_dimensions['E'].width = 12
    # Center align data
    for row in range(2, len(top_towns_export) + 2):
        for col in range(1, 6):
            ws3.cell(row=row, column=col).alignment = Alignment(horizontal='center', vertical='center')
    
    # Sheet 4: All combinations data
    print("  Creating All Combinations sheet...")
    all_combinations_export = coverage_data.sort_values('total_rows', ascending=False)[
        ['MAKE_DESC', 'INV_STATE_ABBRV', 'STATE_TOWN', 'total_rows', 'coverage_rows', 'coverage_percentage']
    ].copy()
    all_combinations_export.columns = ['Make', 'State', 'State, Town', 'Total Rows', 'Coverage Rows', 'Coverage %']
    all_combinations_export['Coverage %'] = all_combinations_export['Coverage %'].round(1)
    all_combinations_export.to_excel(writer, sheet_name='All Combinations', index=False)
    
    ws4 = writer.sheets['All Combinations']
    # Format header
    for cell in ws4[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center')
    # Set column widths
    ws4.column_dimensions['A'].width = 15
    ws4.column_dimensions['B'].width = 10
    ws4.column_dimensions['C'].width = 35
    ws4.column_dimensions['D'].width = 15
    ws4.column_dimensions['E'].width = 15
    ws4.column_dimensions['F'].width = 12
    
    # Highlight top 20 combinations with appropriate colors
    print("  Applying highlighting to All Combinations sheet...")
    for row_idx, row_data in enumerate(all_combinations_export.itertuples(), start=2):
        make = row_data[1]  # Make
        state_town = row_data[3]  # State, Town
        combination = (make, state_town)
        
        in_top_occurrence = combination in top_20_occurrence_set
        in_top_coverage = combination in top_20_coverage_set
        
        if in_top_occurrence and in_top_coverage:
            # Both: Yellow background + bold
            fill = PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid')
            font = Font(bold=True)
        elif in_top_occurrence:
            # Only occurrence: Yellow background
            fill = PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid')
            font = Font(bold=False)
        elif in_top_coverage:
            # Only coverage: Green background
            fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
            font = Font(bold=False)
        else:
            fill = None
            font = None
        
        for col in range(1, 7):
            cell = ws4.cell(row=row_idx, column=col)
            if fill:
                cell.fill = fill
            if font:
                cell.font = font
            cell.alignment = Alignment(horizontal='center', vertical='center')
    
    # Sheet 5: Summary statistics
    print("  Creating Summary sheet...")
    summary_data = {
        'Metric': [
            'Total Rows in Dataset',
            'Rows Used in Analysis',
            'Rows Skipped (null/empty)',
            'Percentage of Rows Skipped',
            'Rows with null INV_STATE_ABBRV',
            'Rows with empty INV_STATE_ABBRV',
            'Rows with null INV_TOWN_NAME',
            'Rows with empty INV_TOWN_NAME',
            'Rows with null MAKE_DESC',
            'Rows with empty MAKE_DESC',
            'Overall Sales Coverage %',
            'Number of Makes',
            'Number of States',
            'Number of (State, Town) tuples',
            'Total Make × (State, Town) Combinations',
            'Combinations with 0% Coverage',
            'Combinations with 100% Coverage',
            'Combinations with Partial Coverage',
            'Top 20 by Occurrence - Yellow Highlight',
            'Top 20 by Coverage % - Green Highlight',
            'In Both Lists - Yellow + Bold',
        ],
        'Value': [
            f"{total_rows:,}",
            f"{valid_rows:,}",
            f"{skipped_rows:,}",
            f"{100*skipped_rows/total_rows:.2f}%",
            f"{null_state:,}",
            f"{empty_state:,}",
            f"{null_town:,}",
            f"{empty_town:,}",
            f"{null_make:,}",
            f"{empty_make:,}",
            f"{100*total_with_coverage/valid_rows:.2f}%",
            f"{unique_makes:,}",
            f"{unique_states:,}",
            f"{unique_towns:,}",
            f"{total_combinations:,}",
            f"{count_0_pct:,} ({100*count_0_pct/total_combinations:.2f}%)",
            f"{count_100_pct:,} ({100*count_100_pct/total_combinations:.2f}%)",
            f"{count_between:,} ({100*count_between/total_combinations:.2f}%)",
            'See All Combinations sheet',
            'See All Combinations sheet',
            'See All Combinations sheet'
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    ws5 = writer.sheets['Summary']
    # Format header
    for cell in ws5[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center')
    # Set column widths
    ws5.column_dimensions['A'].width = 45
    ws5.column_dimensions['B'].width = 30
    # Align data
    for row in range(2, len(summary_df) + 2):
        ws5.cell(row=row, column=1).alignment = Alignment(horizontal='left', vertical='center')
        ws5.cell(row=row, column=2).alignment = Alignment(horizontal='center', vertical='center')

print(f"\n✓ Results exported to: {output_file}")
print(f"\nExcel file contains:")
print(f"  - Sheet 1: Top 20 by Occurrence (Yellow highlighting)")
print(f"  - Sheet 2: Top 20 by Coverage % (Green highlighting)")
print(f"  - Sheet 3: Top Towns (Top 30 by occurrence)")
print(f"  - Sheet 4: All Combinations (with Yellow/Green highlighting)")
print(f"  - Sheet 5: Summary (statistics)")
print(f"\nHighlighting legend:")
print(f"  🟨 Yellow: Top 20 by occurrence")
print(f"  🟩 Green: Top 20 by highest coverage %")
print(f"  🟨 Yellow + Bold: In both top 20 lists")
print("\n✓ Export complete!")


Exporting to Excel with formatting...
  Creating Top 20 by Occurrence sheet...
  Creating Top 20 by Coverage % sheet...
  Creating Top Towns sheet...
  Creating All Combinations sheet...
  Applying highlighting to All Combinations sheet...
  Creating Summary sheet...

✓ Results exported to: sales_coverage_by_make_and_state_town.xlsx

Excel file contains:
  - Sheet 1: Top 20 by Occurrence (Yellow highlighting)
  - Sheet 2: Top 20 by Coverage % (Green highlighting)
  - Sheet 3: Top Towns (Top 30 by occurrence)
  - Sheet 4: All Combinations (with Yellow/Green highlighting)
  - Sheet 5: Summary (statistics)

Highlighting legend:
  🟨 Yellow: Top 20 by occurrence
  🟩 Green: Top 20 by highest coverage %
  🟨 Yellow + Bold: In both top 20 lists

✓ Export complete!
